In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nbeats_keras.model import NBeatsNet as NBeatsKeras
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
#from nbeats_pytorch.model import NBeatsNet as NBeatsPytorch
from keras.optimizers import RMSprop, Adam
import time
from keras.models import load_model
#from target_data_electronic70_7 import target_X, target_y ,test_X, test_y
#from m4databasis21_7 import base_domain,zt_in,zt_out,M4Meta,inputsize,train_12,train_12_y
from sklearn.metrics import mean_squared_error,mean_absolute_error,mean_absolute_percentage_error
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False
from tensorflow.keras.losses import Loss
import tensorflow as tf
#from m4databasis35_7_70_7 import train_35,train_35_y,train_70,train_70_y
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LayerNormalization, MultiHeadAttention, Dropout, Add, Concatenate,Flatten,Reshape
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, Lambda
from tensorflow.keras.callbacks import EarlyStopping

#################################################################################
# loss SMAPE
class SMAPE(Loss):
    def call(self, y_true, y_pred):
        y_pred = tf.reshape(y_pred, tf.shape(y_true))  # 예측 값의 차원을 맞춤
       # y_pred=tf.clip_by_value(y_pred, 1e-10, tf.reduce_max(y_pred))
       # y_true = tf.clip_by_value(y_true, 1e-10, tf.reduce_max(y_true))
        
        numerator = 100 * tf.abs(y_true- y_pred )
        denominator =  (tf.abs(y_true ) + tf.abs(y_pred))/2
        smape =  numerator /  denominator #tf.clip_by_value(denominator, 1e-10, tf.reduce_max(denominator))
        return tf.reduce_mean(smape)

#################################################################################
# loss MASE
class MASE(Loss):
    def __init__(self, training_data, period, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.scale = self.calculate_scale(training_data, period)
    def seasonal_diff(data, period):
        return data[period:] - data[:-period]

    def calculate_scale(self, training_data, period):
        # 주기 차분 계산
        diff = seasonal_diff(training_data, period)
        scale = np.mean(np.abs(diff))
        return scale
    
    def call(self, y_true, y_pred):
        y_pred = tf.reshape(y_pred, tf.shape(y_true))  # 차원 맞추기
        error = tf.abs(y_true - y_pred)
        return tf.reduce_mean(error / self.scale)

def seasonal_diff(data, period):
    return data[period:] - data[:-period]
#################################################################################
# 하이퍼파라미터 인자 설정
def hyperparameter():
    # 1 backcast
    # 2 forecast
    # 3 inputdim
    # 4 outputdim
    # 5 unit
    # 6 bacth size
    return X_train.shape[1],1,y_train.shape[1],y_train.shape[1]

#################################################################################
# nbeats 모델 생성 함수
def build_model(input_timesteps,features,output_timesteps,unit):
    model = Sequential()
    model.add(LSTM(unit, return_sequences=True, input_shape=(input_timesteps, features)))
    #model.add(LSTM(unit, return_sequences=True))
    # Use Lambda layer to select the last 'output_timesteps' outputs
    model.add(Lambda(lambda x: x[:, -output_timesteps:, :]))
    model.add(TimeDistributed(Dense(1)))
    optimizer = Adam(learning_rate=0.001)
    model.compile(loss='mse', optimizer=optimizer)
    return model

#################################################################################
# 부트스트랩 샘플링
# 배깅

def train_bagging_models(num_models, loss_fn , epochs_, patience_,batch_size_,lr):
    models = {}
    input_timesteps,features,output_timesteps,unit = hyperparameter()
    historys = []
    for n in range(num_models):
        K.clear_session()
        model = build_model(input_timesteps,features,output_timesteps,unit)
       # model.set_weights(pretrained_weights)  # 전이 학습 가중치 적용
        optimizer = Adam(learning_rate=lr)
        model.compile(optimizer=optimizer , loss=loss_fn)
        
        # 부트스트랩 샘플링
        #select = np.random.choice(len(X_train), size=len(X_train), replace=True)
        #X_bootstrap = X_train[select]
        #y_bootstrap = y_train[select]
        early_stop = EarlyStopping(monitor='val_loss', patience = patience_, verbose = 1, restore_best_weights=True)
        history = model.fit(X_train, y_train, batch_size = batch_size_,
                  epochs=epochs_, verbose=0, 
                  callbacks=[early_stop],
                 validation_split = 0.2)
        models[f'model_{n+1}'] = model
        historys.append(history)
        #models.append(model)
        print(f"'########################################################Model{n}")
    return models,historys

#################################################################################


#################################################################################

def bagging_predict(models, X):
    predictions = np.array([model.predict(X) for model in models.values()])
    return np.median(predictions, axis=0)

def bagging_predict2(models, X):
    predictions = np.array([model.predict(X) for model in models.values()])
    return predictions

In [4]:
target_X= pd.read_csv("data/solor_train_input_7.csv").iloc[:,(1+24*0):].values
target_y =pd.read_csv("data/solor_train_output_7.csv").iloc[:,1:].values
test_X= pd.read_csv("data/solor_val_input_7.csv").iloc[:,(1+24*0):].values
test_y =pd.read_csv("data/solor_val_output_7.csv").iloc[:,1:].values

X_train=target_X
y_train=target_y

backcast_length = X_train.shape[1]
forecast_length = y_train.shape[1]

model_num = 1

mase_models = train_bagging_models(model_num, MASE(y_train,24),300,10,8,0.001)
mape_models = train_bagging_models(model_num,'mape',300,10,8,0.001)
smape_models = train_bagging_models(model_num, SMAPE(),300,10,8,0.001)
pred1,_=smape_models
pred2,_=mase_models
pred3,_=mape_models
a1= bagging_predict(pred1, test_X)
a2= bagging_predict(pred2, test_X)
a3= bagging_predict(pred3, test_X)

print(mean_squared_error(test_y.flatten(),a1.flatten()),mean_absolute_error(test_y.flatten(),a1.flatten()))
print(mean_squared_error(test_y.flatten(),a2.flatten()),mean_absolute_error(test_y.flatten(),a2.flatten()))
print(mean_squared_error(test_y.flatten(),a3.flatten()),mean_absolute_error(test_y.flatten(),a3.flatten()))

2024-09-06 16:41:16.900604: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-09-06 16:41:16.900645: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ymlee2-desktop): /proc/driver/nvidia/version does not exist
2024-09-06 16:41:16.901323: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Restoring model weights from the end of the best epoch: 57.
Epoch 67: early stopping
'########################################################Model0
Restoring model weights from the end of the best epoch: 8.
Epoch 18: early stopping
'########################################################Model0
Restoring model weights from the end of the best epoch: 30.
Epoch 40: early stopping
'########################################################Model0
12/12 [==============================] - 0s 10ms/step
0.26318839055946364 0.2599580355666313
0.23907690662905057 0.24007488760818146
1.165172431542645 0.594461878259684


In [5]:
target_X= pd.read_csv("data/solor_train_input_5.csv").iloc[:,(1+24*0):].values
target_y =pd.read_csv("data/solor_train_output_5.csv").iloc[:,1:].values
test_X= pd.read_csv("data/solor_val_input_5.csv").iloc[:,(1+24*0):].values
test_y =pd.read_csv("data/solor_val_output_5.csv").iloc[:,1:].values

X_train=target_X
y_train=target_y

backcast_length = X_train.shape[1]
forecast_length = y_train.shape[1]

model_num = 1

mase_models = train_bagging_models(model_num, MASE(y_train,24),300,10,8,0.001)
mape_models = train_bagging_models(model_num,'mape',300,10,8,0.001)
smape_models = train_bagging_models(model_num, SMAPE(),300,10,8,0.001)
pred1,_=smape_models
pred2,_=mase_models
pred3,_=mape_models
a1= bagging_predict(pred1, test_X)
a2= bagging_predict(pred2, test_X)
a3= bagging_predict(pred3, test_X)

print(mean_squared_error(test_y.flatten(),a1.flatten()),mean_absolute_error(test_y.flatten(),a1.flatten()))
print(mean_squared_error(test_y.flatten(),a2.flatten()),mean_absolute_error(test_y.flatten(),a2.flatten()))
print(mean_squared_error(test_y.flatten(),a3.flatten()),mean_absolute_error(test_y.flatten(),a3.flatten()))

Restoring model weights from the end of the best epoch: 20.
Epoch 30: early stopping
'########################################################Model0
Restoring model weights from the end of the best epoch: 12.
Epoch 22: early stopping
'########################################################Model0
Restoring model weights from the end of the best epoch: 30.
Epoch 40: early stopping
'########################################################Model0
12/12 [==============================] - 0s 7ms/step
0.24838947126064972 0.24983518782090847
0.2379673152935491 0.24469310761919746
1.1892831134111048 0.6000721486582357


In [6]:
target_X= pd.read_csv("data/solor_train_input_3.csv").iloc[:,(1+24*0):].values
target_y =pd.read_csv("data/solor_train_output_3.csv").iloc[:,1:].values
test_X= pd.read_csv("data/solor_val_input_3.csv").iloc[:,(1+24*0):].values
test_y =pd.read_csv("data/solor_val_output_3.csv").iloc[:,1:].values

X_train=target_X
y_train=target_y

backcast_length = X_train.shape[1]
forecast_length = y_train.shape[1]

model_num = 1

mase_models = train_bagging_models(model_num, MASE(y_train,24),300,10,8,0.001)
mape_models = train_bagging_models(model_num,'mape',300,10,8,0.001)
smape_models = train_bagging_models(model_num, SMAPE(),300,10,8,0.001)
pred1,_=smape_models
pred2,_=mase_models
pred3,_=mape_models
a1= bagging_predict(pred1, test_X)
a2= bagging_predict(pred2, test_X)
a3= bagging_predict(pred3, test_X)

print(mean_squared_error(test_y.flatten(),a1.flatten()),mean_absolute_error(test_y.flatten(),a1.flatten()))
print(mean_squared_error(test_y.flatten(),a2.flatten()),mean_absolute_error(test_y.flatten(),a2.flatten()))
print(mean_squared_error(test_y.flatten(),a3.flatten()),mean_absolute_error(test_y.flatten(),a3.flatten()))

Restoring model weights from the end of the best epoch: 22.
Epoch 32: early stopping
'########################################################Model0
Restoring model weights from the end of the best epoch: 12.
Epoch 22: early stopping
'########################################################Model0
Restoring model weights from the end of the best epoch: 27.
Epoch 37: early stopping
'########################################################Model0
12/12 [==============================] - 0s 5ms/step
0.27103444103985674 0.2643563491893518
0.24107069477816717 0.2443009433545586
1.1832813881170485 0.5982839716911494
